# Create time stamps for each USGS lidar tile

In [10]:
laz_fn = "/mnt/1.0_TB_VOLUME/sethv/shashank_data/TRAIN_tile_stack_baker_128_global_coreg/pc_laz_prep_full_outputs_USGS_LPC_WA_MtBaker_2015_10UEU8395_LAS_2017/lower_easton3/USGS_LPC_WA_MtBaker_2015_10UEU8395_LAS_2017_32610_first_filt_v1.3.laz"

out_fn = "USGS_LPC_WA_MtBaker_2015_10UEU8395_LAS_2017_32610_first_filt_v1.3_timestamps.tif"

In [11]:
# pipeline = [
#     laz_fn,
#     # Crop in native CRS to remove as many points as possible
#     {"type": "filters.crop", "polygon": polygon_list},
#     {"type": "filters.returns", "groups": "first,only"},
#     # Filter outliers to label as noise points (slow?)
#     {"type": "filters.outlier"},
#     {"type": "filters.range", "limits": "Classification![7:7]"},
#     {"type": "filters.reprojection", "out_srs": out_crs},
#     {"type": "writers.las", "filename": filtered_laz_fn, "compression": "true"},
# ]
res = 1
!pdal translate -i $laz_fn -o $out_fn --writers.gdal.data_type="float64"  --writers.gdal.dimension="GpsTime" --writers.gdal.resolution=$res --writers.gdal.output_type="all" --writers.gdal.gdalopts="COMPRESS=LZW,TILED=YES,BIGTIFF=IF_SAFER" 

In [19]:
import glob
pattern = "/mnt/1.0_TB_VOLUME/sethv/shashank_data/*tile_stack_baker_128_global_coreg/**/lower_easton3/USGS_LPC_WA_MtBaker_2015_*_LAS_2017_32610_first_filt_v1.3.laz"
laz_files = list(glob.glob(pattern))
len(laz_files)

128

In [34]:
timestamps_out_fn = "USGS_LPC_WA_MtBaker_2015_128_timestamps_10m.tif"
!gdal_merge.py -o $timestamps_out_fn USGS_LPC*timestamps.tif

In [35]:
ts = rioxarray.open_rasterio(timestamps_out_fn)
!gdalinfo $timestamps_out_fn

Driver: GTiff/GeoTIFF
Files: USGS_LPC_WA_MtBaker_2015_128_timestamps_10m.tif
Size is 801, 1602
Coordinate System is:
PROJCRS["WGS 84 / UTM zone 10N",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["UTM zone 10N",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",-123,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["Scale factor at natural origin",0.9996,
            SCALEUNIT["unity",1],
            ID["EPSG",8805]],
        PARAMETER["False easting",500000,
            LENGTHUNIT["metre",1],
         

In [54]:
float(ts[0][100][100])

126040971.99259782

In [61]:
import astropy.time

import numpy as np
def convert_gpstime(ts):
    return astropy.time.Time(10**9 + ts, format="gps")

ufunc = np.frompyfunc(convert_gpstime, 1, 1)



In [62]:
# ts_range = ts[1] - ts[0]
ts_day_of_year_min = ufunc(ts[0])

In [47]:
ts_day_of_year_max = [astropy.time.Time(10**9 + x, format="gps") for x in ts[1]]

In [ ]:
import os
for laz_fn in laz_files:
    bn = os.path.basename(laz_fn)
    prefix, _ = os.path.splitext(bn)
    out_fn = prefix + "_timestamps.tif"
    # print(out_fn)
    res = 10
    !pdal translate -i $laz_fn -o $out_fn --writers.gdal.data_type="float64"  --writers.gdal.dimension="GpsTime" --writers.gdal.resolution=$res --writers.gdal.output_type="all" --writers.gdal.gdalopts="COMPRESS=LZW,TILED=YES,BIGTIFF=IF_SAFER" 